In [ ]:
import math
# 数据类是Python的一种对象，用于存储相关的属性数据
# 数据类主要是用于存储数据的类，并且它会自动为你提供常用的特殊方法，例如 __init__、__repr__ 和 __eq__ 等。
from dataclasses import dataclass
from typing import Any, Tuple, Optional
from tqdm.notebook import tqdm

# FairScale是一个用于大规模训练的PyTorch扩展库
# 这里导入的initialize_model_parallel函数用于初始化模型并行环境
import fairscale.nn.model_parallel.initialize as fs_init
import torch
import torch.nn.functional as F
from torch import nn

In [ ]:
# FairScale库的模块，提供了一些用于模型并行的层:

# ① ColumnParallelLinear: 它是一个并行版本的线性（全连接）层，用于在列方向上分割权重矩阵。具体来说，假设我们有一个全连接层，
# 其权重矩阵的维度为 [in_features, out_features]，ColumnParallelLinear 会将 out_features 划分成几个较小的部分，
# 并在多个设备上分别进行计算。这样可以在每个设备上节省内存，从而允许我们训练更大的模型。

# ② RowParallelLinear是FairScale库中的一个类，主要用于实现大型模型的并行训练。它使用了一种称为“行并行”的方法，
# 即把一个大的线性层在最后一个维度（也就是“行”维度）上分割成若干个较小的线性层，然后在不同的设备（例如多个 GPU）上分别进行计算。
# 这样可以降低每个设备的内存占用，从而允许训练更大的模型。

# ③ ParallelEmbedding: 与ColumnParallelLinear类似，这是一个并行版本的嵌入层，用于在列方向上分割嵌入矩阵。假设我们有一个嵌入层，
# 其权重矩阵的维度为[num_embeddings, embedding_dim]，ParallelEmbedding会将embedding_dim划分成几个较小的部分，并在多个设备上分别进行计算。


from fairscale.nn.model_parallel.layers import (
    ColumnParallelLinear,
    RowParallelLinear,
    ParallelEmbedding,
)

In [ ]:
@dataclass
class ModelArgs:
    dim: int = 4096 # 模型的维度
    n_layers: int = 32 # 模型中的层数
    n_headers: int = 32 # 模型中的注意力头数
    n_kv_heads: Optional[int] = None # 模型中的键/值对的头数
    vocab_size: int = -1 # 词汇表的大小
    multiple_of: int = 256 # # 使SwiGLU隐藏层的大小为2的大次幂的倍数
    ffn_dim_multiplier: Optional[float] = None # 前馈神经网络（Feed Forward Network）维度的乘数
    norm_eps: float = 1e-5 # 归一化的epsilon参数
    max_batch_size: int = 32 # 最大批处理大小
    max_seq_len: int = 2048 # 最大序列长度